#PROJECT-1
#"ANAMOLY DETECTION"

In [46]:
import pandas as pd
import numpy as np
import os
import sys
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

In [47]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [48]:
from pyod.utils.utility import standardizer,precision_n_scores
from sklearn.metrics import roc_auc_score

In [49]:
mat_file=["arrhythmia.mat",
         "cardio.mat",
         "glass.mat",
         "ionosphere.mat",
         "letter.mat",
         "lympho.mat",
         "mnist.mat",
         "musk.mat",
         "optdigits.mat",
         "pendigits.mat",
         "pima.mat",
         "satellite.mat",
         "satimage-2.mat",
         "shuttle.mat",
         "vertebral.mat",
         "vowels.mat",
         "wbc.mat"]

In [50]:
data=loadmat(mat_file[0])
print(data.items())


dict_items([('__header__', b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue May 17 11:53:12 2016'), ('__version__', '1.0'), ('__globals__', []), ('X', array([[ 75. ,   0. , 190. , ...,   2.9,  23.3,  49.4],
       [ 56. ,   1. , 165. , ...,   2.1,  20.4,  38.8],
       [ 54. ,   0. , 172. , ...,   3.4,  12.3,  49. ],
       ...,
       [ 36. ,   0. , 166. , ...,   1. , -44.2, -33.2],
       [ 32. ,   1. , 155. , ...,   2.4,  25. ,  46.6],
       [ 78. ,   1. , 160. , ...,   1.6,  21.3,  32.8]])), ('y', array([[1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [51]:
type(data["X"]),data["X"].shape

(numpy.ndarray, (452, 274))

In [52]:
type(data["y"]),data["y"].shape

(numpy.ndarray, (452, 1))

EXPLORING THE MAT-FILES


In [53]:
df_columns=["Data","Samples","Dimensions","Outlier Perc","PCA","MCD","OCSVM","LOF","CBLOF","KNN","ABOD","HBOS","IForest","FeatureBagging"]
time_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
roc_df=pd.DataFrame(columns=df_columns)

from time import time
random_state=np.random.RandomState(50)
for file in mat_file:
    print("PROCESSING... ",file)
    mat=loadmat(file)
    x=mat["X"]
    y=mat["y"].ravel()
    outliers_fraction=np.count_nonzero(y)/len(y)
    outliers_percentage=round(outliers_fraction*100,ndigits=4)
    roc_list=[file[:-4],x.shape[0],x.shape[1],outliers_percentage]
    prn_list=[file[:-4],x.shape[0],x.shape[1],outliers_percentage]
    time_list=[file[:-4],x.shape[0],x.shape[1],outliers_percentage]
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=random_state)
    x_train_norm,x_test_norm=standardizer(x_train,x_test)
    
    
    classifiers={
        
        "Principal Componenet Analysis": PCA(contamination=outliers_fraction,random_state=random_state),
        "Minimum Covariance Determinant": MCD(contamination=outliers_fraction,random_state=random_state),
        "One Class SVM": OCSVM(contamination=outliers_fraction),
        "Local Outlier Factor": LOF(contamination=outliers_fraction),
        "Cluster-Based Local Outlier Factor": CBLOF(contamination=outliers_fraction,random_state=random_state,check_estimator=False),
        "K Nearest Neighbor":KNN(contamination=outliers_fraction),
        "Angle based Outlier Detection":ABOD(contamination=outliers_fraction),
        "Histogram Based Outlier Detection": HBOS(contamination=outliers_fraction),
        "Isolation Forest":IForest(contamination=outliers_fraction),
        "Feature Bagging": FeatureBagging(contamination=outliers_fraction,random_state=random_state)
           
        
    }
    for clf_name, clf in classifiers.items():
        t0=time()
        clf.fit(x_train_norm)
        test_scores=clf.decision_function(x_test_norm)
        t1=time()
        duration=round(t1-t0,ndigits=4)
        time_list.append(duration)
        
        roc=round(roc_auc_score(y_test,test_scores),ndigits=4)
        prn=round(precision_n_scores(y_test,test_scores),ndigits=4)
        
        print("{clf_name}\n ROC:{roc}\n precision @ rank n:{prn}\n execution time:{duration}s".format(clf_name=clf_name, roc=roc,prn=prn,duration=duration))
        
        roc_list.append(roc)
        prn_list.append(prn)
    
    
    temp_df=pd.DataFrame(time_list).transpose()
    temp_df.columns=df_columns
    time_df=pd.concat([time_df,temp_df],axis=0)
    
    temp_df=pd.DataFrame(prn_list).transpose()
    temp_df.columns=df_columns
    prn_df=pd.concat([prn_df,temp_df],axis=0)
     
    temp_df=pd.DataFrame(roc_list).transpose()
    temp_df.columns=df_columns
    roc_df=pd.concat([roc_df,temp_df],axis=0)
        
        
        

PROCESSING...  arrhythmia.mat
Principal Componenet Analysis
 ROC:0.7166
 precision @ rank n:0.2857
 execution time:0.1251s


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant
 ROC:0.7477
 precision @ rank n:0.2857
 execution time:1.3783s
One Class SVM
 ROC:0.7155
 precision @ rank n:0.2857
 execution time:0.0591s
Local Outlier Factor
 ROC:0.7172
 precision @ rank n:0.2143
 execution time:0.0981s


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Cluster-Based Local Outlier Factor
 ROC:0.7184
 precision @ rank n:0.2857
 execution time:0.1832s
K Nearest Neighbor
 ROC:0.7359
 precision @ rank n:0.2857
 execution time:0.1321s
Angle based Outlier Detection
 ROC:0.7196
 precision @ rank n:0.3571
 execution time:0.3143s
Histogram Based Outlier Detection
 ROC:0.7547
 precision @ rank n:0.3571
 execution time:0.1361s
Isolation Forest
 ROC:0.7213
 precision @ rank n:0.3571
 execution time:0.7587s
Feature Bagging
 ROC:0.7061
 precision @ rank n:0.2143
 execution time:0.8048s
PROCESSING...  cardio.mat
Principal Componenet Analysis
 ROC:0.9487
 precision @ rank n:0.5556
 execution time:0.007s


...

Cluster-Based Local Outlier Factor
 ROC:0.3979
 precision @ rank n:0.0
 execution time:0.0901s
K Nearest Neighbor
 ROC:0.4262
 precision @ rank n:0.0909
 execution time:0.016s
Angle based Outlier Detection
 ROC:0.4784
 precision @ rank n:0.2727
 execution time:0.1001s
Histogram Based Outlier Detection
 ROC:0.3353
 precision @ rank n:0.0
 execution time:0.005s
Isolation Forest
 ROC:0.4247
 precision @ rank n:0.0909
 execution time:0.4995s
Feature Bagging
 ROC:0.5216
 precision @ rank n:0.1818
 execution time:0.0621s
PROCESSING...  vowels.mat
Principal Componenet Analysis
 ROC:0.6762
 precision @ rank n:0.1875
 execution time:0.004s
Minimum Covariance Determinant
 ROC:0.6529
 precision @ rank n:0.0
 execution time:1.2312s
One Class SVM
 ROC:0.7914
 precision @ rank n:0.4375
 execution time:0.0811s
Local Outlier Factor
 ROC:0.955
 precision @ rank n:0.5
 execution time:0.0671s


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Cluster-Based Local Outlier Factor
 ROC:0.8999
 precision @ rank n:0.4375
 execution time:0.1561s
K Nearest Neighbor
 ROC:0.9654
 precision @ rank n:0.5625
 execution time:0.1351s
Angle based Outlier Detection
 ROC:0.9745
 precision @ rank n:0.625
 execution time:0.6586s
Histogram Based Outlier Detection
 ROC:0.7044
 precision @ rank n:0.1875
 execution time:0.009s
Isolation Forest
 ROC:0.7594
 precision @ rank n:0.1875
 execution time:0.6246s
Feature Bagging
 ROC:0.9592
 precision @ rank n:0.5625
 execution time:0.5295s
PROCESSING...  wbc.mat
Principal Componenet Analysis
 ROC:0.985
 precision @ rank n:0.6667
 execution time:0.004s
Minimum Covariance Determinant
 ROC:0.979
 precision @ rank n:0.6667
 execution time:0.1081s
One Class SVM
 ROC:0.985
 precision @ rank n:0.6667
 execution time:0.01s
Local Outlier Factor
 ROC:0.994
 precision @ rank n:0.6667
 execution time:0.015s


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Cluster-Based Local Outlier Factor
 ROC:0.985
 precision @ rank n:0.6667
 execution time:0.1271s
K Nearest Neighbor
 ROC:0.985
 precision @ rank n:0.6667
 execution time:0.0331s
Angle based Outlier Detection
 ROC:0.9369
 precision @ rank n:0.0
 execution time:0.1691s
Histogram Based Outlier Detection
 ROC:0.991
 precision @ rank n:0.6667
 execution time:0.016s
Isolation Forest
 ROC:0.988
 precision @ rank n:0.6667
 execution time:0.5195s
Feature Bagging
 ROC:0.994
 precision @ rank n:0.6667
 execution time:0.1351s


In [54]:
time_df

,Data,Samples,Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,ABOD,HBOS,IForest,FeatureBagging
0,arrhythmia,452,274,14.6018,0.1251,1.3783,0.0591,0.0981,0.1832,0.1321,0.3143,0.1361,0.7587,0.8048
0,cardio,1831,21,9.6122,0.007,1.0419,0.1712,0.1832,0.2783,0.3053,1.058,0.014,0.7808,1.3653
0,glass,214,9,4.2056,0.005,0.1361,0.006,0.009,0.0911,0.015,0.0971,0.005,0.4955,0.0621
0,ionosphere,351,33,35.8974,0.005,0.1121,0.012,0.013,0.0841,0.03,0.1632,0.018,0.5225,0.1291
0,letter,1600,32,6.25,0.011,2.0549,0.1441,0.1962,0.2182,0.2703,0.8507,0.021,0.6997,1.4013
0,lympho,148,18,4.0541,0.003,0.0601,0.002,0.005,0.0771,0.01,0.0711,0.01,0.4794,0.052
0,mnist,7603,100,9.2069,0.2152,5.6133,7.7184,10.4277,1.4694,11.2956,13.7109,0.1001,3.5393,72.1385
0,musk,3062,166,3.1679,0.2061,22.0086,1.7066,3.244,0.6016,3.6264,4.3571,0.1211,2.4062,22.0256
0,optdigits,5216,64,2.8758,0.0711,2.3009,2.3382,2.3182,0.6556,2.6915,4.5633,0.0571,1.8297,17.4604
0,pendigits,6870,16,2.2707,0.0171,3.6053,2.0259,0.9109,0.3964,1.012,3.3712,0.017,1.2261,5.9042


In [55]:
prn_df

,Data,Samples,Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,ABOD,HBOS,IForest,FeatureBagging
0,arrhythmia,452,274,14.6018,0.2857,0.2857,0.2857,0.2143,0.2857,0.2857,0.3571,0.3571,0.3571,0.2143
0,cardio,1831,21,9.6122,0.5556,0.4815,0.4815,0.2037,0.5556,0.4074,0.2222,0.4815,0.4259,0.2037
0,glass,214,9,4.2056,0.3333,0,0.3333,0,0.3333,0.3333,0.3333,0,0,0.3333
0,ionosphere,351,33,35.8974,0.5676,0.8649,0.6486,0.7297,0.6486,0.8378,0.8108,0.4054,0.6486,0.7027
0,letter,1600,32,6.25,0.1351,0.2432,0.1622,0.4595,0.2703,0.4054,0.4054,0.1351,0.1351,0.4595
0,lympho,148,18,4.0541,1,1,1,1,1,1,1,1,1,1
0,mnist,7603,100,9.2069,0.3832,0.2664,0.3925,0.3131,0.4252,0.4299,0.3832,0.1168,0.3271,0.3131
0,musk,3062,166,3.1679,1,1,1,0.2593,1,0.2963,0.1111,1,1,0.2593
0,optdigits,5216,64,2.8758,0,0,0,0.0667,0,0,0,0.2889,0.0667,0.0667
0,pendigits,6870,16,2.2707,0.2,0.05,0.225,0.1,0.25,0.1,0.075,0.225,0.175,0.1


In [56]:
roc_df

,Data,Samples,Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,ABOD,HBOS,IForest,FeatureBagging
0,arrhythmia,452,274,14.6018,0.7166,0.7477,0.7155,0.7172,0.7184,0.7359,0.7196,0.7547,0.7213,0.7061
0,cardio,1831,21,9.6122,0.9487,0.8665,0.9383,0.5982,0.875,0.7942,0.6406,0.8682,0.9141,0.6092
0,glass,214,9,4.2056,0.6505,0.7742,0.5968,0.7796,0.7742,0.8011,0.7312,0.7473,0.7043,0.8172
0,ionosphere,351,33,35.8974,0.7728,0.9671,0.8026,0.8813,0.8559,0.9181,0.9287,0.6063,0.8308,0.8766
0,letter,1600,32,6.25,0.5285,0.8091,0.5945,0.8781,0.7671,0.8686,0.8636,0.6022,0.6589,0.8546
0,lympho,148,18,4.0541,1,1,1,1,1,1,1,1,1,1
0,mnist,7603,100,9.2069,0.8529,0.8461,0.8537,0.7021,0.851,0.8432,0.7657,0.571,0.7934,0.695
0,musk,3062,166,3.1679,1,1,1,0.5718,1,0.8072,0.1734,1,1,0.5641
0,optdigits,5216,64,2.8758,0.5331,0.3694,0.5249,0.4769,0.7677,0.3911,0.4781,0.8617,0.7918,0.4728
0,pendigits,6870,16,2.2707,0.9156,0.8033,0.9098,0.4664,0.9538,0.7258,0.6474,0.8779,0.9327,0.4611
